Imports necessary libraries for data analysis, preprocessing, modeling, and visualization.

Creates an output directory named 'output' if it doesn't exist. This will be used to store output files or visualizations generated later in the code.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.cross_decomposition import CCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
from functions import utils




output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)
 

<span style="font-size: 0.75em;">

# Financial Literacy Data Processing

This script processes and cleans datasets from two different years (2017 and 2020) related to financial literacy. The final output is a prepared DataFrame ready for further analysis or modeling. Below is a step-by-step explanation of the process:

## 1. Load Datasets and Compare Columns
The script begins by loading two datasets, one from 2017 and the other from 2020. It then compares the columns of these datasets:
- If the columns match, a message `"PASS_20_17"` is printed.
- If the columns do not match, the differences are identified and displayed.

## 2. Merge DataFrames and Initial Cleaning
Next, the two datasets are merged into a single DataFrame. Some initial data cleaning is performed:
- A new column `key` is added as a unique identifier, which is then set as the index of the DataFrame.
- The redundant `id` column is dropped.

## 3. Preprocess `qprod1_d`, `qprod2`, `qk4`, `qk5`, and `qf10_*` Variables
Certain variables require specific preprocessing:
- Missing values in specific columns (`qprod1_d` and `qprod2`) are filled with predefined values (`99` and `-99`, respectively).
- The variables `qk4` and `qk5` are recoded, grouping the answers of variables qk4 and qk5, imposing 1 on all incorrect answers.
- Variables that start with `qf10_` have invalid values (`-97`, `-99`) replaced with the mode (most frequent value) of each column to get an integer variable.

## 4. Define and Encode Variables
The script then categorizes the variables into three types:
- **Binary variables**: Variables with binary (0/1) responses.
- **Integer variables**: Variables with integer responses, including the `qf10_*` variables.
- **Continuous variables**: Variables with continuous numeric responses (e.g., `pesofitc`).

The remaining columns are treated as **categorical variables**. These categorical variables are encoded using `OneHotEncoder`, which converts them into binary (0/1) features.

## 5. Create and Finalize the Final DataFrame
The final DataFrame (`db_final`) is created by combining the binary, integer, and encoded categorical variables. Any remaining missing values are filled with `0`.

## 6. Check for Missing Values
Lastly, the script checks if there are any remaining missing values in the final DataFrame:
- If missing values are detected, a warning message is printed.
- If there are no missing values, a success message is displayed.

This structured approach ensures that the data is clean, consistent, and ready for any further analysis or machine learning tasks.

In [3]:
# 1. Load Datasets and Compare Columns
df_2017 = pd.read_csv("data/Dataset - Financial literacy/Financia_literacy_2017.csv")
df_2020 = pd.read_csv("data/Dataset - Financial literacy/Financia_literacy_2020.csv")

if (df_2020.columns == df_2017.columns).all():
    print("PASS_20_17")
else:
    print("FAIL_20_17")
    print("df_20: YES , df_17: NO", set(df_2020.columns) - set(df_2017.columns))
    print("df_17: YES , df_20: NO", set(df_2017.columns) - set(df_2020.columns))

# 2. Merge DataFrames and creates a new index
db = pd.concat([df_2017, df_2020], axis=0, ignore_index=True)
db['key'] = range(len(db))
db.set_index('key', inplace=True)
db.drop(columns=["id"], inplace=True)

# 3. Preprocess qprod1_d, qprod2, qk4, qk5, and qf10_* Variables
db.fillna({"qprod1_d": 99, "qprod2": -99}, inplace=True)

db['qk4'] = np.where(db['qk4'].isin([-97, -99, 0]), db['qk4'], 1)
db['qk5'] = np.where(db['qk5'].isin([-97, -99, 102]), db['qk5'], 1)

qf10_cols = [col for col in db.columns if col.startswith("qf10_")]
for col in qf10_cols:
    mode_value = db[col].mode()[0]
    db[col] = db[col].replace([-97, -99], mode_value)

# 4. Define Variables Type and Encode Variables Categorical ones
binary_variables = [
    "qd1", "SM", "qd12", "qprod1c_1", "qprod1c_2", "qprod1c_3", "qprod1c_5", "qprod1c_6", "qprod1c_7",
    "qprod1c_8", "qprod1c_10", "qprod1c_11", "qprod1c_12", "qprod1c_14", "qprod1c_99", "qf3_1", "qf3_3", "qf3_4",
    "qf3_6", "qf3_7", "qf3_8", "qf3_99", "qf9_1", "qf9_10", "qf9_2", "qf9_3", "qf9_4", "qf9_5", "qf9_6", "qf9_7",
    "qf9_8", "qf9_9", "qf9_99", "qprod3_1", "qprod3_2", "qprod3_3", "qprod3_4", "qprod3_5", "qprod3_6", "qprod3_7",
    "qprod3_8", "qprod3_9", "qprod3_10", "qprod3_11", "qprod3_12", "qprod3_13", "qprod3_14", "qprod3_15", "qprod3_16",
    "qprod3_17", "qprod3_18", "qprod3_99", "qf12_1_a", "qf12_1_b", "qf12_1_c", "qf12_2_d", "qf12_3_e", "qf12_3_f",
    "qf12_3_g", "qf12_4_k", "qf12_4_l", "qf12_5_m", "qf12_5_o", "qf12_6_p", "qf12_6_q", "qf12_7_r", "qf12_97", "qf12_99"
]
integer_variables = ["qd5b", "qd7"] + qf10_cols
continuous_variables = ["pesofitc"]

categorical_variables = db.columns.difference(binary_variables + integer_variables + continuous_variables)

encoder = OneHotEncoder(drop='first')
encoded_cols = encoder.fit_transform(db[categorical_variables])
encoded_df = pd.DataFrame(encoded_cols.toarray(), columns=encoder.get_feature_names_out(categorical_variables))

# 5. Create and Finalize the Final DataFrame
db_final = pd.concat([db[binary_variables + integer_variables], encoded_df], axis=1).fillna(0)

# 6. Check for Missing Values
if db_final.isna().any().any():
    print("Missing values exist.")
else:
    print("No missing values.")

PASS_20_17
No missing values.


<span style="font-size: 1em;">

### Variable Selection for Financial Literacy Analysis

In this analysis, I have specifically defined and selected variables based on the questions from the questionnaire that are directly used to calculate the financial literacy scores.
\\
These variables are categorized into three key components: **knowledge**, **behavior**, and **attitude**. 

In [5]:
knowledge_score = ["qk3_2","qk3_3","qk3_-97","qk3_1", #qk3
                       "qk4_1","qk4_-97","qk4_0", #qk4
                       "qk5_1","qk5_-97","qk5_102", #qk5
                       "qk6_2","qk6_3","qk6_4","qk6_-97","qk6_1", #qk6
                       "qk7_1_0","qk7_1_-97","qk7_1_1", #qk7_1
                       "qk7_2_0","qk7_2_-97","qk7_2_1", #qk7_2
                       "qk7_3_0","qk7_3_-97","qk7_3_1",] #qk7_3

behevioral_score =  ["qf1_2","qf1_3","qf1_1", #qf1
                        "qf2_0","qf2_1", #qf2
                        "qf3_1","qf3_3","qf3_4","qf3_6","qf3_7","qf3_8","qf3_99", #qf3
                        "qf10_1", #qf10_1
                        "qf10_4", #qf10_4
                        "qf10_6", #qf10_6
                        "qf10_7", #qf10_7
                        "qprod2_2.0","qprod2_3.0","qprod2_4.0","qprod2_1.0", #qprod2
                        "qprod3_1","qprod3_2","qprod3_3","qprod3_4","qprod3_5","qprod3_6","qprod3_7","qprod3_8","qprod3_9","qprod3_10","qprod3_11","qprod3_12","qprod3_13","qprod3_14","qprod3_15","qprod3_16","qprod3_17","qprod3_18","qprod3_99", #qprod3
                        "qf12_1_a","qf12_1_b","qf12_1_c","qf12_2_d","qf12_3_e","qf12_3_f","qf12_3_g","qf12_4_k","qf12_4_l","qf12_5_m","qf12_5_o","qf12_6_p","qf12_6_q","qf12_7_r","qf12_97","qf12_99" #qf12
                        ]

attitude_score = ["qf10_2","qf10_3","qf10_8"] #qf10